In [30]:
%reset
import re
import requests
import codecs
import time
import nltk

import pandas as pd

from bs4 import BeautifulSoup as bs

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

import yfinance as yf


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Probleemomschrijving
Het doel is om een pipeline te creeën die data ophaalt, transformeert en laad om vervolgens gebruikt te kunnen worden om de prijs van de ETF VUSA te voorspellen. Hierbij gaan wij artikelen van investopedia en yahoo finance scrapen en hier sentiment analysis op toepassen. Ook halen wij de historische prijsdata op van de verschillende aandelen die zich in de ETF bevinden. En als laatst halen wij de historische prijsdata op van VUSA zelf op en voegen wij alles samen tot 1 DataFrame. Deze pipeline kan elke dag gerund worden om zo nieuwe artikelen op te halen en deze toe te voegen aan de dataset, zo krijg je een steeds beter bruikbare dataset.

# Domeinonderzoek
Vusa (IE00B3XXRP09) is een ETF die beheert wordt door Vanguard group (Fondsbeheerder|NOT,Disclosed|Vanguard S&P 500 UCITS ETF (EUR)|ISIN:IE00B3XXRP09, n.d.). Een ETF is een aandeel wat eigenlijk bestaat uit meerdere aandelen, obligaties of andere effecten die op de markt verhandelt worden (Wat Zijn ETF’s | Educatie | BlackRock, n.d.). VUSA specifiek volgt de marktindex van de SP500, dat zijn aandelen van de 500 grootste bedrijven van de Verenigde Staten. 

De SP500 geeft een goed beeld van de amerikaanse markt, aangezien het veel verschillende sectoren en aandelen van grote bedrijven binnen deze sectoren bevat. SP500 heeft daarom een unieke ligging in de aandelenmarkt, omdat het zo groot kunnen er duidelijke veranderingen in de amerikaanse markt worden gemeten naar aanleiding van de SP500. Omdat de SP500 zo'n goed beeld geeft van de amerikaanse markt worden veel beleggingen hiermee vergeleken om te kijken hoe goed het gaat. (S&P 500 | Wat Is De S&P 500? | Beleggingswiki - Semmie.nl, n.d.).

# Vereiste informatie
Wij willen willen artikelen scrapen om daar sentiment analysis op toe te passen. Wij zijn van mening dat dit belangrijke data is aangezien veel aandelen verhandeld worden met bots. Deze bots gebruiken ook sentiment analyse om te kijken of een aandeel gekocht moet worden, hierdoor kan de prijs beinvloed worden door het sentiment van de artikelen over een bedrijf (SWOCC, 2020). 

Ook maken wij gebruik van de prijsgeschiedenis van de aandelen die zich in het ETF bevinden, dit omdat de ETF een verzameling is van alle bedrijven en dus zullen de individuele bedrijven de ETF ook beinvloeden. Deze data komt uit een database die elke dag dat de beurzen open zijn geupdate wordt.

Als laatst maken wij gebruik van een data over de prijsgeschiedenis van de SP500 zelf, de patronen die hierin zitten kunnen namelijk een beeld geven van hoe de toekomst van de SP500 en de ETF's die dit tracken (zoals ons aandeel: VUSA) eruit kan zien.




# Databronnen en formaat ruwe data
- https://www.investopedia.com/markets-news-4427704 (html)
- https://finance.yahoo.com/ (html)
- https://github.com/ranaroussi/yfinance (dataframe)
- https://datahub.io/core/s-and-p-500 (csv)
- https://datahub.io/core/s-and-p-500-companies (csv)



# passende technieken voor het extraheren
Voor het extraheren van de data maken wij gebruik van 4 verschillende technieken.

## get requests
Wij gebruiken get requests voor het ophalen van pagina's die niet dynamisch geladen zijn en geen cookies nodige hebben om de html van de pagina te kunnen halen. Wij doen een get request doormiddel van de requests library en zetten de content vervolgens om naar een beautifulsoup object. Hierdoor kunnen we makkelijk door de html spitten om de juiste data te vinden.

Dit wordt gedaan bij deze bronnen:
- **investopedia:**<br>
Hier worden de hoofdpagina opgehaald en de urls van de artikelen eruit gehaald. Deze urls worden daarna ook met get requests opgehaald om zo de artikelen eruit te kunnen halen
- **datahub.io:**<br>
Wij gebruiken datahub 2 keer, 1 keer voor de prijsgeschiedenis van de SP500 en 1 keer om de verschillende ticker symbolen van de SP500 bedrijven te achterhalen. Voor beide pagina's wordt een get request gebruikt om de downloadknop te zoeken en dan vervolgens is er nog een get requests om de url van de downloadknop op te halen en de CSV data die daar opgehaald wordt in een CSV bestand te schrijven.

## Selenium (headless)
Wij gebruiken een headless selenium browser om pagina's te scrapen die ofwel een knop hebben waar op geklikt moet worden of dynamisch geladen worden. Om het simpel te zeggen: pagina's waar we niet de juiste informatie kunnen ophalen zonder een actie uit te voeren. 

We gebruiken headers die moeten zorgen dat wij minder op een bot lijken en meer op een persoon die de website bezoekt via een normale browser.

Dit wordt gedaan bij deze bronnen:
- **Yahoo finance (hoofdpagina)**<br>
Voor de website van yahoo finance gebruiken wij selenium omdat deze dynamisch geladen wordt. Dit houdt in dat wij eerst de hoofdpagina moeten laden en daar helemaal naar beneden moeten scrollen voordat de html alle data bevat van alle artikelen. Ook is het handig aangezien we dan niet met cookies hoeven te werken, we kunnen gewoon op de knop klikken die niet-essentiele cookies weigert en we worden toegelaten tot de website.
- **Yahoo finance (artikelen)**<br>
Als eenmaal alle urls van de artikelen opgehaald zijn gebruiken wij selenium om deze te laden. Dit doen we omdat als een artikel te lang is moet er op een knop gedrukt worden die de rest van het artikel toont. Wij kijken doormiddel van een try-except of die knop er is en zo ja dan wordt er op geklikt.

## yfinance
Voor de API gebruiken wij yfinance, dit is een publieke tool die ontwikkeld is om makkelijk met de API van yahoo finance te praten. 

Wij gebruiken yfinance om de verschillende prijsgeschiedenissen van de bedrijven in de SP500 op te halen. Deze komen uit de database van yahoo finance en worden elke dag dat de beurzen open zijn geupdate.

# Webscraping:
Voor het webscrapen hebben wij zoals eerder genoemd get requests en een headless selenium browser gebruikt. We zullen hier mee toelichting geven over hoe de webscraping wordt uigevoerd.

- **investopedia**<br>
Om te beginnen doen wij een get request naar de URL van investopedia (https://www.investopedia.com/markets-news-4427704). Hier maken wij een beautifulsoup object van om vervolgens zoeken wij alle urls op doormiddel van .find_all('a', class_='card'). Dit zijn de a tags die naar de artikelen leiden die wij willen hebben. We zetten de href attribuut en de text van deze tag apart in een dataframe met de naam investopedia_df. <br><br>Dan begint het scrapen van de artikelen. Wij doen get requests naar de url's met de artikelen en gebruiken .find('div', class_='article-content').find_all('p') om alle paragrafen van de artikelen uit het beautifulsoup object te halen die de html bevat. Deze worden samengevoegd tot 1 artikel en toegevoegd aan investopedia_df<br><br>Daarnaast heeft investopedia ook ticker symbolen in de artikelen staan van de bedrijven waar het over gaat. De ticker symbolen zijn allemaal ook a-tags omdat ze leiden naar de investopedia pagina van dat bedrijf. Wij scrapen de symbolen door dit te doen: [a_tag.text for a_tag in p.find_all('a') if re.search('widgetsymbol', a_tag['href'])], zo kunnen we alle text van de a-tags ophalen als de a-tag naar een webpagina leidt met "widgetsymbol" in de URL. Dit wordt gebruikt door investopedia voor alle pagina's van bedrijven met ticker symbolen. Ook deze symbolen worden toegevoegd aan investopedia_df<br><br>Als laatst word de investopedia_df gefilterd op artikelen waarvan de symbolen die erin voorkomen ook voorkomen in de SP500.<br><br>

- **Yahoo finance**<br>
Bij yahoo finance maken we gebruik van selenium, we laden de hoofdpagina, weigeren niet-essentiele cookies en scrollen naar beneden doormiddel van een ActionChains. In deze ActionChains kan je acties zetten die de browser voor je uitvoert. Wij zetten doormiddel van een for-loop 1500 keer in deze ActionChains dat er gescrolled moet worden: .scroll_by_amount(0, 500). Dit zorgt ervoor dat de webpagina altijd helemaal naar beneden gescrolled wordt voordat er gekeken wordt welke artikelen er allemaal zijn.<br><br>Vervolgens worden alle a-tags uit de html gehaald doormiddel van .find_elements(By.TAG_NAME, 'a'). Van deze a-tags worden de text, hrefs en css classes opgeslagen in een dataframe en dit wordt gefilterd op de css class van de a-tags die naar artikelen verwijzen en of de url "/news/" bevat. zie code: df[df['url'].str.contains('/news/') & df['classes'].str.contains('js-content-viewer')].<br><br>Nadat we alle urls van de artikelen in een dataframe hebben worden de artikelen gescraped. Dit doen we met een nieuwe selenium browser die eerst weer alle niet-essentiele cookies weigert en dan alle artikelen 1 voor 1 laadt. Op elke pagina wordt er gekeken of er een knop is die artikelen verder toont die ingedrukt moet worden en als die er is wordt het ingedrukt. Dit wordt gedaan in een try-except:.find_element(By.CLASS_NAME, 'collapse-button').click() die probeert op de knop te drukken en als dat niet gaat wordt pass gerunt dus gebeurd er niks. Dit zorgt ervoor dat we alle artikelen volledig kunnen zien. Deze artikelen worden vervolgens gescraped door .find_element(By.CLASS_NAME, 'caas-body').find_elements(By.TAG_NAME, 'p') te runnen om zo de paragrafen van de artikelen te verkrijgen. Dit wordt gecheckt op inhoud en samengevoegd op deze manier: [p.text for p in article_p_elements if p.text!='']
<br><br>

# API:
Voor de API maken wij gebruik van de Yahoo finance API doormiddel van yfinance. yfinance is een tool die het makkelijk maakt om informatie van yahoo finance te krijgen. Deze library doet dit door calls te maken naar yahoo finance waar je normaal gesproken autorisatie voor nodig hebt. calls worden gemaakt naar de REST API van yahoo finance, bijv. https://query2.finance.yahoo.com/v7/finance/options/ticker_symbol_example. De library maakt dit allemaal meer pythonic en makkelijker doordat je met deze library alleen yf.download(symbol_list, group_by="ticker") hoeft te gebruiken. Dan krijg je meteen een dataframe terug met alle data van alle tickers die in je symbol_list staan. Dit is ook hoe wij de data ophalen uit de API. <br><br>Wij gebruiken de eerder vernoemde CSV van datahub.io met alle ticker symbolen in de SP500 en zetten dit in een lijst om zo allemaal te downloaden. De API geeft standaard een dataframe met multiIndex waar de eerste rij van de index de ticker is en de 2e rij de soort informatie van de ticker. Bij de 2e rij staan er voor elke ticker de volgende informatie in het DataFrame: Open, High, Low, Close, adjusted Close en Volume. Dit zijn veelvoorkomende gegevens die aangeven hoe een aandeel zich die dag heeft bewogen en zijn daarom erg belangrijk voor ons.

# Opschonen van de data / uitleg text voorbewerking

### text data
- **Preprocessing**<br>
Voor de text data is alle text uit de html gehaald doormiddel van beautifulsoup of selenium, dit is vervolgens schoongemaakt. Dit schoonmaken wordt gedaan door alle speciale tekens eruit te halen en alleen woorden en witte characters over te houden. Dit bereiken wij door regex te gebruiken re.sub(r'[^\w\s]', '', text). Deze manier om precies te zijn.<br><br>Als alle text schoon is moet deze worden getokenized zodat ze bruikbaar zijn in de sentiment analyse. Dit tokenizen doen wij met de NLTK library. Ook halen we de stopwoorden eruit om zo te zorgen dat de text gereduceert wordt tot alleen belangrijke dingen. De stopwoorden die wij eruit halen zijn te vinden in nltk.corpus.stopwords.words('english'). Nu we alleen nog maar belangrijke text hebben is het van belang te zorgen dat alle text uniform wordt, dit doen we door lemmatization met NLTK. Lemmatization zorgt ervoor dat de woorden allemaal teruggebracht worden naar een vorm die makkelijk te herkennen en te gebruiken is voor sentiment analyse. <br><br>

- **feature engineering**<br>
Na het preprocessen van de text wordt deze door de SentimentIntensityAnalyzer gehaald. Deze functie van NLTK geeft sentiment analyse scores terug van de texten. Wij halen de positiviteitsscore en negativiteitsscore hieruit en kijken naar de basisstatistieken van deze scores van alle artikelen die gescraped zijn. Zo hebben we de gemiddelde, standaard deviatie, de kwartielen, het maximum en het minimum van de positiviteit en de negativiteit. Deze zetten we in een dataframe met 1 rij zodat deze toegevoegd kan worden aan de uiteindelijke dataframe bij vandaag.

### API data
Deze data komt al netjes en schoon in de vorm van een dataframe.

### Datahub.io
Deze data downloaden wij als CSV en dit zorgt ervoor dat als wij deze CSV vervolgens gebruiken dat dit makkelijk en netjes in een dataframe gezet kan worden.

### Uiteindelijke dataframe
Na het transformeren en samenvoegen van alle data doen wij nog een aantal preprocessing stappen op het uiteindelijke dataframe om deze bruikbaar te maken voor machine learning.

- **preprocessing**<br>
Wij vervangen alle NaN waardes met 0 zodat het dataframe alleen nog maar numerieke waardes bevat, dit zodat het in een keer gebruikt kan worden voor machine learning. Ook verwijderen wij de kolom: "Date" die overgebleven is van het samenvoegen van de verschillende dataframes tot 1 dataframe. 


In [31]:
class Pipeline:
    """
    Pipeline to get data relating to the stock vusa that tracks the SP500. Includes sentiment analysis from articles,
    price history of stocks that are in the ETF and price history of vusa itself. 
    The class follows the Extract, Transform, Load functionality.
    """
    def __init__(self):
        
        self.HEADERS = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'en-US,en;q=0.9,nl;q=0.8',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}

        self.DATA_LOCATION = 'Datasets/'

        self.HOMEPAGE_URL = 'https://www.investopedia.com/markets-news-4427704'

        self.VUSA_DOWNLOAD_URL = 'https://datahub.io/core/s-and-p-500-companies'
        self.SP_DOWNLOAD_URL = 'https://datahub.io/core/s-and-p-500'
        self.DATAHUB_URL = 'https://datahub.io'

        self.YAHOO_URL = 'https://finance.yahoo.com/'

        self.OPTIONS = Options()
        self.OPTIONS.add_argument('--headless')
        self.OPTIONS.add_argument('--disable-gpu') 
        
        nltk.download(['punkt', 'wordnet', 'stopwords', 'omw-1.4', 'vader_lexicon'], quiet=True)
        
        
    def get_response(self, url, headers):
        r = requests.get(url, headers=headers)
        soup = bs(r.content)
        return soup
    
    def get_datahub_csv(self, URL, filename):
        soup = self.get_response(URL, self.HEADERS)
        download_url = soup.find_all('table')[1].find_all('a')[1]['href']
        download_url = self.DATAHUB_URL + download_url
        with open(self.DATA_LOCATION + filename, 'wb') as f:
            f.write(requests.get(download_url).content)
    
    def get_yahoo_news_articles_df(self):
        yahoo_news_driver = webdriver.Chrome(options=self.OPTIONS)

        yahoo_news_driver.get(self.YAHOO_URL)
        time.sleep(1)
        yahoo_news_driver.find_element(By.CLASS_NAME, 'reject-all').click()
        actions = ActionChains(yahoo_news_driver)
        for i in range(1500):
            actions.scroll_by_amount(0, 500)
        actions.perform()
        actions.reset_actions()

        links = self.get_article_links(yahoo_news_driver)
        df = pd.DataFrame({'title': [link.text for link in links], 
                           'url': [link.get_attribute('href') for link in links], 
                           'classes': [link.get_attribute('class') for link in links]})

        yahoo_news_driver.quit()
        df = df[df['url'] \
                .str.contains('/news/') & df['classes'].str.contains('js-content-viewer')]\
                .drop_duplicates(subset='url', keep='last')\
                .reset_index(drop=True)
        return df
    
    def scrape_articles(self, urls):
        articles = []
        raw_articles = []
        yahoo_driver = webdriver.Chrome(options=self.OPTIONS)
        yahoo_driver.get(self.YAHOO_URL)
        yahoo_driver.find_element(By.CLASS_NAME, 'reject-all').click()

        for index, url in enumerate(urls):
            yahoo_driver.get(url)
            try:
                yahoo_driver.find_element(By.CLASS_NAME, 'collapse-button').click()
            except:
                pass
            article_p_elements = yahoo_driver.find_element(By.CLASS_NAME, 'caas-body').find_elements(By.TAG_NAME, 'p')
            article_by_paragraph = [p.text for p in article_p_elements if p.text!='']

            raw_article = [bs(element.get_attribute('outerHTML'), "html.parser") for element in article_p_elements]
            article = " ".join(article_by_paragraph)
            clean_article = self.clean_text(article)
            articles.append(clean_article)
            raw_articles.append(raw_article)
            self.progress_bar(index+1, len(urls))


        yahoo_driver.quit()

        return articles, raw_articles
    
    def download_yahoo_data(self, file_path):
        """
        Downloadt gegevens van Yahoo Finance voor tickers die zijn vermeld in het opgegeven CSV-bestand.

        Parameters:
        - file_path (str): Bestandspad naar het CSV-bestand met ticker-symbolen.

        Returns:
        - pd.DataFrame: Gefilterd DataFrame met gedownloade gegevens.
        """
        df = pd.read_csv(file_path)
        symbol_list = df.iloc[:, 0].tolist()

        download = yf.download(symbol_list, group_by="ticker")
        data = download.copy()

        filtered_data = data.dropna(axis=1, how='all')
        filtered_data.columns = filtered_data.columns.remove_unused_levels()

        remaining_tickers = list(filtered_data.columns.levels[0])
        missing_tickers = list(set(symbol_list) - set(remaining_tickers))

        return filtered_data    
    
    def get_card_titles(self):
        titles = []
        a_tags = self.soup.find_all('a', class_='card')
        for i in range(len(a_tags)):
            titles.append(a_tags[i].span.text)
        return titles

    def get_card_urls(self):
        urls = []
        a_tags = self.soup.find_all('a', class_='card')
        for i in range(len(a_tags)):
            urls.append(a_tags[i]['href'])
        return urls
    
    def get_url_content(self, urls):
        articles = []
        raw_articles = []
        for url in urls:
            soup = self.get_response(url, self.HEADERS)
            paragraphs = soup.find('div', class_='article-content').find_all('p')
            article = ''.join([p.get_text(separator=' ') for p in paragraphs])
            raw_articles.append(paragraphs)
            articles.append(self.clean_text(article))

        return articles, raw_articles
    
    def get_symbols(self, raw_articles):
        symbol_list = []
        for p_list in raw_articles:
            article_symbols = []
            for p in p_list:
                symbols = [a_tag.text for a_tag in p.find_all('a') if re.search('widgetsymbol', a_tag['href'])]
                if symbols:
                    for symbol in symbols:
                        article_symbols.append(symbol)
            symbol_list.append(article_symbols)
        return symbol_list
    
    def clean_text(self, text):
        return re.sub(r'[^\w\s]', '', text)
            
    def get_article_links(self, driver):
        links = driver.find_elements(By.TAG_NAME, 'a')
        return links
    
    def progress_bar(self, current, total, bar_length=20):
        fraction = current / total

        arrow = int(fraction * bar_length - 1) * '-' + '>'
        padding = int(bar_length - len(arrow)) * ' '

        ending = '\n' if current == total else '\r'

        print(f'Progress: [{arrow}{padding}] {int(fraction*100)}%', end=ending)

    def clean_and_tokenize(self, articles):
        lemmatizer = WordNetLemmatizer()

        tokenized_articles = []
        for article in articles:
            tokenized_article = nltk.word_tokenize(article)
            tokenized_article = [token for token in tokenized_article if token not in stopwords.words('english')]
            tokenized_article = [lemmatizer.lemmatize(token) for token in tokenized_article]

            tokenized_articles.append(tokenized_article)

        return tokenized_articles
    
    def combine_data(self, api_df, investopedia_df, yahoo_df, time_data):
        df1 = investopedia_df.drop(['symbols'], axis=1)
        df2 = yahoo_df.drop('classes', axis=1)
        text_df = pd.concat([df1, df2])
        text_df = text_df.reset_index(drop=True)
        
        # tokenize the articles and add them to a DataFrame
        text_df['tokenized_article'] = self.clean_and_tokenize(text_df['article'])
        
        # apply sentiment analysis and add it to the DataFrame
        analyzer = SentimentIntensityAnalyzer()
        text_df['positivity_score'] = [analyzer.polarity_scores(article)['pos'] for article in text_df['article']]
        text_df['negativity_score'] = [analyzer.polarity_scores(article)['neg'] for article in text_df['article']]
        
        # get statistics from positive and negative sentiment analysis
        pos = text_df.describe().T[:1].reset_index(drop=True)
        neg = text_df.describe().T[1:].reset_index(drop=True)
        
        # rename columns in pos and neg so they can be concattinated
        pos.columns = ['pos_' + col for col in pos.columns]
        neg.columns = ['neg_' + col for col in neg.columns]
        
        # concattinate the positve and negative sentiment analysis statistics to a 1 row DataFrame
        pos_neg_row = pd.concat([pos, neg], axis=1)
        
        api_df = api_df.iloc[::-1].reset_index(drop=False, names='Date')
        
        api_df.columns = ['_'.join(col).strip() for col in api_df.columns.values]
        
        combined_df = pd.concat([api_df, pos_neg_row], axis=1)
        
        combined_df['Date_'] = combined_df['Date_'].astype(str)
        time_data['Date'] = time_data['Date'].astype(str)

        final_df = combined_df.merge(time_data, how='left', left_on='Date_', right_on='Date')
        
        return final_df

    def extract(self):
        # get site data
        self.soup = self.get_response(self.HOMEPAGE_URL, self.HEADERS)

        # download csv data
#         self.get_datahub_csv(self.VUSA_DOWNLOAD_URL, 'vusa_holdings.csv')
#         self.get_datahub_csv(self.SP_DOWNLOAD_URL, 'time_data.csv')
       
        # get csv data
        self.vusa_df = pd.read_csv(self.DATA_LOCATION + 'vusa_holdings.csv')
        self.time_data_df = pd.read_csv(self.DATA_LOCATION + 'time_data.csv')

        # get yahoo articles with titles, urls and css classes
        self.yahoo_news_df = self.get_yahoo_news_articles_df()
        self.yahoo_news_df = self.yahoo_news_df[:2]
    
        # scrape news articles from yahoo article urls
        self.yahoo_news_articles, self.raw_yahoo_news_articles = self.scrape_articles(self.yahoo_news_df['url'])
        
        # get api data
        self.api_df = self.download_yahoo_data('Datasets/vusa_holdings.csv')
    
    
    def transform(self):        
        # filter site data
        titles = self.get_card_titles()
        urls = self.get_card_urls()
        
        # transform filtered data to DataFrame(investopedia_df)
        investopedia_df = pd.DataFrame({'url': urls, 'title': titles})
        
        # get articles from urls
        articles, raw_articles = self.get_url_content(investopedia_df['url'])
        
        # filter symbols from the vusa_df csv data
        symbols_in_vusa = self.vusa_df['Symbol'].tolist()

        # add scraped articles to DataFrame
        self.yahoo_news_df['article'] = self.yahoo_news_articles
 
        # get symbols from article
        symbols = self.get_symbols(raw_articles)
        
        # add articles and symbols to DataFrame(investopedia_df)
        investopedia_df['article'] = articles
        investopedia_df['symbols'] = symbols
        
        # filter DataFrame(investopedia_df) to only have symbols that occur in the symbols_in_vusa
        investopedia_df = investopedia_df[investopedia_df['symbols'].apply(lambda symbols: any(symbol in symbols_in_vusa for symbol in symbols))]
        
        # combine DataFrames
        self.df = self.combine_data(self.api_df, investopedia_df, self.yahoo_news_df, self.time_data_df)
        self.df = self.df.fillna(0)
        self.df = self.df.drop('Date', axis=1)
          
    def load(self, filename):
        self.df.to_csv(filename)
    
    def ETL(self, filename):
        self.extract()
        self.transform()
        self.load(filename)
        

# Het volledige proces
Er is al veel uitgelegd in vorige beschrijvingen, toch vinden wij het belangrijk om alles nog 1 keer van begin tot eind te beschrijven zodat alle stappen duidelijk zijn die gevolgd worden.

## ETL
Wij maken in dit project gebruik van ETL (Extract, transform, load). Hiervoor hebben wij ook een ETL methode aangemaakt onderaan de class, hier worden de Extract, transform en load methodes aangeroepen die vervolgens de stappen uivoeren die nodig zijn.


## Extract
In de Extract methode halen wij alle ruwe data uit de beschikbare bronnen. 

Het begint met het aanroepen van de get_response methode met de URL voor investopedia (https://www.investopedia.com/markets-news-4427704), get_response doet hier een get request en zet de html om in een beautifulsoup object.

vervolgens werden de csv bestanden gedownload van datahub.io maar aangezien dit niet meer beschikbaar is, is dit uigecomment. En in plaats hiervan worden de gedownloade bestanden ingelezen als dataframes en in de variabelen: vusa_df en time_data_df gezet.

Dan wordt de yahoo finance website gescraped doormiddel van de get_yahoo_news_articles_df methode, deze methode weigert de niet-essentiele cookies en scrollt dan door de yahoo_finance website(https://finance.yahoo.com/) om zo alle URLs op te halen van de artikelen die op de website staan. Dit doen we doormiddel van een headless selenium browser, aangezien de website dynamisch geladen word en zonder scrollen niet alle URLs opgehaald kunnen worden. Deze URLs worden in een DataFrame gezet met de titels van de artikelen en css klassen van de artikelen op de website. We halen de artikelen en css klassen ook op om duidelijk te kunnen zien of alles goed is gegaan en problemen makkelijk op te lossen als het niet zou gaan zoals wij verwachten.

Dan hebben we alle URLs van de artikelen maar de artikelen zelf hebben we nog niet, dus dat is het volgende wat wij doen. 
Dit doen we met de methode scrape_articles waar wij de URLs als parameter meegeven. Deze methode klikt eerst op het weigeren van niet-essentiele cookies en laadt vervolgens 1 voor 1 de verschillende URLs. Sommige van de artikelen zijn groter en yahoo finance laadt dan niet het gehele artikel zien, hiervoor moet eerst op een knop gedrukt worden met de class 
'collapse-button'. Wij maken gebruik van een try, except om deze knop in te drukken wat zorgt voor robuustheid zodat alle artikelen goed geladen kunnen worden. vervolgens worden de artikelen opgehaald en schoongemaakt doormiddel van de regex functie die in de methode clean_text gebruikt wordt. scrape_articles geeft ook de rauwe html van de artikelen terug om zo makkelijk te kunnen zien of alles volgens verwachting is verlopen en fouten makkelijk op te lossen.

Als laatst word yfinance API gebruikt om de prijsgeschiedenis te downloaden, dit wordt gedaan in de download_yahoo_data methode. In deze methode wordt ook de locatie van het bestand met de tickers van de bedrijven in de SP500 meegegeven zodat de methode dit kan gebruiken om te weten welke prijsdata gedownload moet worden. In dit bestand staan de tickers van de bedrijven in de SP500 en dit wordt omgezet naar een lijst en gebruikt voor de finance API om te weten wat te downloaden. Dit wordt gedownload en automatisch in een DataFrame gezet. Ook geeft yfinance automatisch aan welke downloads niet gelukt zijn, dit is goed voor de robuustheid aangezien we dan makkelijk kunnen zien waar het wellicht misgaat. Wij hebben een aantal downloads die sowieso mislukken, dit zijn downloads van bedrijven die niet meer verhandelt worden. Een goed voorbeeld hiervan is facebook, facebook wordt niet meer verhandelt aangezien het bedrijf is veranderd naar meta.

Ook worden de symbolen uit vusa_df gefilterd en deze worden in lijst symbols_in_vusa gezet. 


## Transform
Nadat alle ruwe data is geëxtraheerd gaan we de data transformeren zodat het bruikbaar is voor machine learning. We voegen alle data samen, schonen dit op en vervangen lege waardes. 

We beginnen met het beautifulsoup object van de webpagina van investopedia(https://www.investopedia.com/markets-news-4427704). Met de methodes get_card_titles en get_card_urls halen wij de titels en URLs van de artikelen op vanuit de webpagina. De webpagina van investopedia is niet dynamisch geladen dus kunnen wij alle artikelen uit het beautifulsoup object halen. De titels en URLs worden beide opgehaald om makkelijk te kunnen checken of alle artikelen meegenomen zijn al worden er opmerkelijkheden gespot. Deze titels en URLs worden in een dataframe gezet met de naam investopedia_df. Nu we de URLs hebben gebruiken we de functie get_url_content om de artikelen van de website te scrapen doormiddel van beautifulsoup.
De artikelen worden teruggegeven als ruwe artikelen en schone artikelen die schoongemaakt zijn door de clean_text methode.

De ruwe artikelen worden ook gebruikt om de symbolen van de bedrijven te filteren die genoemd worden in het artikel, dit wordt gedaan met de get_symbols methode. Alle artikelen en symbolen worden toegevoegd aan de investopedia_df zodat daar alles instaat van investopedia. vervolgens word de DataFrame gefilterd op de artikelen die symbolen bevat die ook in symbols_in_vusa voorkomt.

Daarna worden de artikelen van yahoo finance die eerder opgehaald zijn bij het extraheren toegevoegd aan de yahoo_news_df. 

Nu hebben we alle data in 4 verschillende DataFrames staan:
- api_df: Hier staat de gedownloade prijsdata van de bedrijven in die voorkomen in de SP500
- investopedia_df: Hier staan de artikelen van investopedia in, gefilterd op of de bedrijven die genoemt zijn in de artikelen voorkomen in de bedrijven van de SP500
- yahoo_news_df: Hier staan de artikelen in van de yahoo finance website
- time_data_df: Hier staat de prijsgeschiedenis van de SP500 zelf in.

Deze 4 Dataframes worden samengevoegd in de combine_data methode, in deze methode wordt de laatste processing gedaan en alles samengevoegd.

De combine_data methode worden eerst de investopedia_df en yahoo_news_df gefilterd en samengevoegt tot 1 text dataframe genaamd text_df. De artikelen uit de text_df worden schoongemaakt, getokenized, gefilterd en gelemmatized in de clean_and_tokenize methode en toegevoegd als nieuwe kolom aan de text_df. Dan wordt doormiddel van de SentimentIntensityAnalyzer de positieve en negatieve sentiment uit de artikelen gehaald en deze worden apart toegevoegd als kolommen in de text_df. Vervolgens moeten we de data zo transformeren dat het allemaal numeriek wordt en klaar om toegevoegd aan de uiteindelijke dataframe. Dit doen we door de basisstatistieken van de positieve en negatieve sentimentscores op te halen en samen te voegen tot een DataFrame van 1 rij. Nu hebben wij een DataFrame met de statistieken van de sentimentscores van vandaag.

Vervolgens voegen wij de statistieken van de sentimentscores van de text toe bij vandaag in de api_df. Deze wordt dan vervolgens gemerged met de time_data op de key met de datums. Zo hebben we dus nu de prijsgeschiedenis van alle bedrijven in de SP500, de prijsgeschiedenis van de SP500 zelf en de statistieken van de sentimentscores van vandaag. Als dit elke dag gerunt word dan worden langzaam steeds meer rijen met statistieken van sentimentscores van alle dagen toegevoegd.

Als laatst worden de missende waardes vervangen met fillna zodat het bruikbaar wordt voor machine learning.

## Load
In de load functie word er voor gezorgd dat de data makkelijk opgehaald en gebruikt kan worden voor machine learning.

Wij doen dit door de DataFrame op te slaan in een CSV bestand wat later makkelijk gemimporteerd en gebruikt kan worden.

In [34]:
pipeline = Pipeline()

In [35]:
pipeline.ETL('Data.csv')

Progress: [------------------->] 100%
[*********************100%%**********************]  505 of 505 completed


27 Failed downloads:
['FRC', 'TWTR', 'RE', 'BRK.B', 'CERN', 'INFO', 'DRE', 'ANTM', 'DISCA', 'CTXS', 'ATVI', 'XLNX', 'WLTW', 'NLSN', 'FISV', 'ABC', 'FBHS', 'KSU', 'BLL', 'DISCK', 'PKI', 'SIVB', 'PBCT', 'NLOK', 'FB', 'VIAC']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1925-02-26 -> 2024-02-03)')


In [36]:
pipeline.df

,Date_,HCA_Open,HCA_High,HCA_Low,HCA_Close,HCA_Adj Close,HCA_Volume,BRO_Open,BRO_High,BRO_Low,...,neg_max,SP500,Dividend,Earnings,Consumer Price Index,Long Interest Rate,Real Price,Real Dividend,Real Earnings,PE10
0,2024-02-02,313.000000,313.000000,307.149994,309.420013,309.420013,1280100.0,77.860001,78.410004,77.650002,...,0.16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-02-01,303.350006,314.820007,303.350006,314.660004,314.660004,1739800.0,77.110001,77.879997,75.790001,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-01-31,303.420013,309.329987,302.390015,304.899994,304.899994,2555800.0,78.540001,78.769997,77.480003,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-01-30,300.000000,304.950012,296.119995,301.589996,301.589996,2483200.0,77.199997,78.330002,77.160004,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-01-29,282.420013,287.190002,281.609985,286.730011,286.730011,1742800.0,76.940002,77.209999,76.169998,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15624,1962-01-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15625,1962-01-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15626,1962-01-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15627,1962-01-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Referentielijst
- Fondsbeheerder|NOT,Disclosed|Vanguard S&P 500 UCITS ETF (EUR)|ISIN:IE00B3XXRP09. (n.d.). https://www.morningstar.nl/nl/etf/snapshot/snapshot.aspx?id=0P0000YXKB&tab=4&InvestmentType=FE
- Wat zijn ETF’s | Educatie | BlackRock. (n.d.). BlackRock. https://www.blackrock.com/be/individual/nl/educatie/etfs-uitgelegd#Wat-zijn-ETF'
- S&P 500 | Wat is de S&P 500? | Beleggingswiki - Semmie.nl. (n.d.). Semmie. https://semmie.nl/wiki/sp-500/#:~:text=De%20S%26P%20500%20is%20een,ontwikkeling%20van%20de%20financi%C3%ABle%20markt.
- SWOCC. (2020, June 8). De invloed van mediaberichtgeving op beurskoersen - SWOCC. https://www.swocc.nl/kennisbank-item/de-invloed-van-mediaberichtgeving-op-beurskoersen/
